In [ ]:

# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES
# TO THE CORRECT LOCATION (/kaggle/input) IN YOUR NOTEBOOK,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

import os
import sys
from tempfile import NamedTemporaryFile
from urllib.request import urlopen
from urllib.parse import unquote, urlparse
from urllib.error import HTTPError
from zipfile import ZipFile
import tarfile
import shutil

CHUNK_SIZE = 40960
DATA_SOURCE_MAPPING = 'cuhk-face-sketch-database-cufs:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F911391%2F1544960%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240731%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240731T121115Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D52927113435af07e97d48204d127609dfbeec27cb0dd81f5a378bf78af79250fbd8f6124c3615be863648d323652356d88602122e0c3eae809a434b5827edbbf9f698daaead8d362e3c1f5eda8f760c4b258bb6e1ee73c9acfbebf97eb830557ca5818a93c98449a9e7c2d2244cd39145fc33372ba1943f79d75c3293f10df0bb4346819bf08e651e60648c460918b1ff815f0c6605398dce48114e22f0ecfd68c3a09c36285003eea96bf0cbc3fe2c14f775575a4eeae23bc665ca15850d334f00d0431825bd5bb004bae5212743518e31c20a9d211ced7efc61f7565fa4523afec95696c17314611071b73eda6fe8f1b106235a10ebb1192f9af98029d91f6'

KAGGLE_INPUT_PATH='/kaggle/input'
KAGGLE_WORKING_PATH='/kaggle/working'
KAGGLE_SYMLINK='kaggle'

!umount /kaggle/input/ 2> /dev/null
shutil.rmtree('/kaggle/input', ignore_errors=True)
os.makedirs(KAGGLE_INPUT_PATH, 0o777, exist_ok=True)
os.makedirs(KAGGLE_WORKING_PATH, 0o777, exist_ok=True)

try:
  os.symlink(KAGGLE_INPUT_PATH, os.path.join("..", 'input'), target_is_directory=True)
except FileExistsError:
  pass
try:
  os.symlink(KAGGLE_WORKING_PATH, os.path.join("..", 'working'), target_is_directory=True)
except FileExistsError:
  pass

for data_source_mapping in DATA_SOURCE_MAPPING.split(','):
    directory, download_url_encoded = data_source_mapping.split(':')
    download_url = unquote(download_url_encoded)
    filename = urlparse(download_url).path
    destination_path = os.path.join(KAGGLE_INPUT_PATH, directory)
    try:
        with urlopen(download_url) as fileres, NamedTemporaryFile() as tfile:
            total_length = fileres.headers['content-length']
            print(f'Downloading {directory}, {total_length} bytes compressed')
            dl = 0
            data = fileres.read(CHUNK_SIZE)
            while len(data) > 0:
                dl += len(data)
                tfile.write(data)
                done = int(50 * dl / int(total_length))
                sys.stdout.write(f"\r[{'=' * done}{' ' * (50-done)}] {dl} bytes downloaded")
                sys.stdout.flush()
                data = fileres.read(CHUNK_SIZE)
            if filename.endswith('.zip'):
              with ZipFile(tfile) as zfile:
                zfile.extractall(destination_path)
            else:
              with tarfile.open(tfile.name) as tarfile:
                tarfile.extractall(destination_path)
            print(f'\nDownloaded and uncompressed: {directory}')
    except HTTPError as e:
        print(f'Failed to load (likely expired) {download_url} to path {destination_path}')
        continue
    except OSError as e:
        print(f'Failed to load {download_url} to path {destination_path}')
        continue

print('Data source import complete.')


In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import os
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from PIL import Image
import torch.nn as nn
import torch.optim as optim
import numpy as np
import matplotlib.pyplot as plt
from torch.optim.lr_scheduler import StepLR

# Fixed hyperparameters
BATCH_SIZE = 1
EPOCHS = 100
LEARNING_RATE = 2e-4

# Fixed paths for Kaggle
IMAGE_DIR = '/kaggle/input/cuhk-face-sketch-database-cufs/photos'
SKETCH_DIR = '/kaggle/input/cuhk-face-sketch-database-cufs/sketches'
RESULTS_DIR = '/kaggle/working/results'

# Create directory if it doesn't exist
os.makedirs(RESULTS_DIR, exist_ok=True)

class ImageSketchDataset(Dataset):
    def __init__(self, image_dir, sketch_dir, transform=None):
        self.image_dir = image_dir
        self.sketch_dir = sketch_dir
        self.transform = transform
        self.image_files = sorted(os.listdir(image_dir))
        self.sketch_files = sorted(os.listdir(sketch_dir))

    def __len__(self):
        return len(self.image_files)

    def __getitem__(self, idx):
        image_path = os.path.join(self.image_dir, self.image_files[idx])
        sketch_path = os.path.join(self.sketch_dir, self.sketch_files[idx])
        image = Image.open(image_path).convert('RGB')
        sketch = Image.open(sketch_path).convert('RGB')

        if self.transform:
            image = self.transform(image)
            sketch = self.transform(sketch)

        return image, sketch

# Data augmentation
transform = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.Resize((256, 256)),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

dataset = ImageSketchDataset(image_dir=IMAGE_DIR,
                             sketch_dir=SKETCH_DIR,
                             transform=transform)

dataloader = DataLoader(dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=2)

class ResidualBlock(nn.Module):
    def __init__(self, in_features):
        super(ResidualBlock, self).__init__()
        self.conv_block = nn.Sequential(
            nn.Conv2d(in_features, in_features, 3, padding=1),
            nn.InstanceNorm2d(in_features),
            nn.ReLU(inplace=True),
            nn.Conv2d(in_features, in_features, 3, padding=1),
            nn.InstanceNorm2d(in_features)
        )

    def forward(self, x):
        return x + self.conv_block(x)

class Generator(nn.Module):
    def __init__(self, num_residual_blocks=9):
        super(Generator, self).__init__()

        # Initial convolution block
        model = [
            nn.Conv2d(3, 64, 7, padding=3),
            nn.InstanceNorm2d(64),
            nn.ReLU(inplace=True)
        ]

        # Downsampling
        in_features = 64
        out_features = in_features * 2
        for _ in range(2):
            model += [
                nn.Conv2d(in_features, out_features, 3, stride=2, padding=1),
                nn.InstanceNorm2d(out_features),
                nn.ReLU(inplace=True)
            ]
            in_features = out_features
            out_features = in_features * 2

        # Residual blocks
        for _ in range(num_residual_blocks):
            model += [ResidualBlock(in_features)]

        # Upsampling
        out_features = in_features // 2
        for _ in range(2):
            model += [
                nn.ConvTranspose2d(in_features, out_features, 3, stride=2, padding=1, output_padding=1),
                nn.InstanceNorm2d(out_features),
                nn.ReLU(inplace=True)
            ]
            in_features = out_features
            out_features = in_features // 2

        # Output layer
        model += [nn.Conv2d(64, 3, 7, padding=3), nn.Tanh()]

        self.model = nn.Sequential(*model)

    def forward(self, x):
        return self.model(x)

class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()

        def discriminator_block(in_filters, out_filters, normalize=True):
            layers = [nn.Conv2d(in_filters, out_filters, 4, stride=2, padding=1)]
            if normalize:
                layers.append(nn.InstanceNorm2d(out_filters))
            layers.append(nn.LeakyReLU(0.2, inplace=True))
            return layers

        self.model = nn.Sequential(
            *discriminator_block(3, 64, normalize=False),
            *discriminator_block(64, 128),
            *discriminator_block(128, 256),
            *discriminator_block(256, 512),
            nn.Conv2d(512, 1, 4, padding=1)
        )

    def forward(self, x):
        return self.model(x)

def weights_init(m):
    classname = m.__class__.__name__
    if classname.find('Conv') != -1:
        nn.init.normal_(m.weight.data, 0.0, 0.02)
    elif classname.find('BatchNorm') != -1:
        nn.init.normal_(m.weight.data, 1.0, 0.02)
        nn.init.constant_(m.bias.data, 0)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

generator_g = Generator().to(device)
generator_f = Generator().to(device)
discriminator_x = Discriminator().to(device)
discriminator_y = Discriminator().to(device)

generator_g.apply(weights_init)
generator_f.apply(weights_init)
discriminator_x.apply(weights_init)
discriminator_y.apply(weights_init)

criterion_GAN = nn.MSELoss()
criterion_cycle = nn.L1Loss()
criterion_identity = nn.L1Loss()

optimizer_g_g = optim.Adam(generator_g.parameters(), lr=LEARNING_RATE, betas=(0.5, 0.999))
optimizer_g_f = optim.Adam(generator_f.parameters(), lr=LEARNING_RATE, betas=(0.5, 0.999))
optimizer_d_x = optim.Adam(discriminator_x.parameters(), lr=LEARNING_RATE, betas=(0.5, 0.999))
optimizer_d_y = optim.Adam(discriminator_y.parameters(), lr=LEARNING_RATE, betas=(0.5, 0.999))

scheduler_g_g = StepLR(optimizer_g_g, step_size=10, gamma=0.1)
scheduler_g_f = StepLR(optimizer_g_f, step_size=10, gamma=0.1)
scheduler_d_x = StepLR(optimizer_d_x, step_size=10, gamma=0.1)
scheduler_d_y = StepLR(optimizer_d_y, step_size=10, gamma=0.1)

def train_step(real_x, real_y):
    real_x = real_x.to(device)
    real_y = real_y.to(device)

    # Train Generators
    optimizer_g_g.zero_grad()
    optimizer_g_f.zero_grad()

    # Identity loss
    same_y = generator_g(real_y)
    loss_identity_y = criterion_identity(same_y, real_y)
    same_x = generator_f(real_x)
    loss_identity_x = criterion_identity(same_x, real_x)

    # GAN loss
    fake_y = generator_g(real_x)
    pred_fake_y = discriminator_y(fake_y)
    loss_gan_g = criterion_GAN(pred_fake_y, torch.ones_like(pred_fake_y))

    fake_x = generator_f(real_y)
    pred_fake_x = discriminator_x(fake_x)
    loss_gan_f = criterion_GAN(pred_fake_x, torch.ones_like(pred_fake_x))

    # Cycle loss
    recovered_x = generator_f(fake_y)
    loss_cycle_x = criterion_cycle(recovered_x, real_x)

    recovered_y = generator_g(fake_x)
    loss_cycle_y = criterion_cycle(recovered_y, real_y)

    # Total loss
    loss_g = loss_gan_g + loss_gan_f + (loss_cycle_x + loss_cycle_y) * 10 + (loss_identity_x + loss_identity_y) * 5

    loss_g.backward()
    optimizer_g_g.step()
    optimizer_g_f.step()

    # Train Discriminator X
    optimizer_d_x.zero_grad()

    pred_real_x = discriminator_x(real_x)
    loss_real_x = criterion_GAN(pred_real_x, torch.ones_like(pred_real_x))

    pred_fake_x = discriminator_x(fake_x.detach())
    loss_fake_x = criterion_GAN(pred_fake_x, torch.zeros_like(pred_fake_x))

    loss_d_x = (loss_real_x + loss_fake_x) * 0.5
    loss_d_x.backward()
    optimizer_d_x.step()

    # Train Discriminator Y
    optimizer_d_y.zero_grad()

    pred_real_y = discriminator_y(real_y)
    loss_real_y = criterion_GAN(pred_real_y, torch.ones_like(pred_real_y))

    pred_fake_y = discriminator_y(fake_y.detach())
    loss_fake_y = criterion_GAN(pred_fake_y, torch.zeros_like(pred_fake_y))

    loss_d_y = (loss_real_y + loss_fake_y) * 0.5
    loss_d_y.backward()
    optimizer_d_y.step()

    return loss_g.item(), loss_d_x.item(), loss_d_y.item()

def train(dataloader, epochs):
    for epoch in range(epochs):
        for i, (real_x, real_y) in enumerate(dataloader):
            loss_g, loss_d_x, loss_d_y = train_step(real_x, real_y)
            if i % 100 == 0:
                print(f'Epoch {epoch+1}/{epochs}, Step {i}, Loss G: {loss_g:.4f}, Loss D_X: {loss_d_x:.4f}, Loss D_Y: {loss_d_y:.4f}')

        scheduler_g_g.step()
        scheduler_g_f.step()
        scheduler_d_x.step()
        scheduler_d_y.step()

        # Generate and save images
        test_input, test_sketch = next(iter(dataloader))
        generate_and_save_images(generator_g, test_input, test_sketch, epoch, 'cyclegan')

def generate_and_save_images(generator, test_input, test_sketch, epoch, filename):
    generator.eval()
    with torch.no_grad():
        prediction = generator(test_input.to(device))
    generator.train()

    test_sketch = test_sketch[0].cpu().permute(1, 2, 0) * 0.5 + 0.5
    prediction = prediction[0].cpu().permute(1, 2, 0) * 0.5 + 0.5

    # Save only the generated image
    plt.figure(figsize=(5, 5))
    plt.imshow(prediction)
    plt.axis('off')
    plt.savefig(os.path.join(RESULTS_DIR, f'{filename}_generated_epoch_{epoch}.png'), bbox_inches='tight', pad_inches=0)
    plt.close()

    # Create comparison graph of sketch and generated image
    plt.figure(figsize=(10, 5))
    plt.subplot(1, 2, 1)
    plt.imshow(test_sketch)
    plt.title('Real Sketch')
    plt.axis('off')

    plt.subplot(1, 2, 2)
    plt.imshow(prediction)
    plt.title('Generated Sketch')
    plt.axis('off')

    plt.savefig(os.path.join(RESULTS_DIR, f'{filename}_comparison_epoch_{epoch}.png'))
    plt.close()

if __name__ == "__main__":
    print(f"Using device: {device}")
    print(f"Starting training for {EPOCHS} epochs")
    train(dataloader, epochs=EPOCHS)
    print("Training completed")
